In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from splinter import Browser
import time

In [20]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. 

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "html.parser")
mars = {}

In [4]:
# Assign the text to variables that you can reference later.

mars['news_title'] = soup.find('div', class_='content_title').get_text().strip()
mars['news_p']= soup.find("div", class_="article_teaser_body").get_text().strip()
mars

{'news_title': "What's Mars Solar Conjunction, and Why Does It Matter?",
 'news_p': 'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.'}

In [21]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string 
# to a variable called `featured_image_url`.

url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/assets/images/logo_nasa_trio_black@2x.png'
browser.visit(url2)
full_image_click= browser.find_by_id('full_image')
full_image_click.click()
time.sleep(3)
browser.click_link_by_partial_text('more info')
html = browser.html
soup = bs(html, "html.parser")

In [22]:
# Make sure to find the image url to the full size `.jpg` image.
# Make sure to save a complete url string for this image.
image_path = soup.find('img', class_='main_image').get('src')
mars['featured_image_url'] = 'https://www.jpl.nasa.gov' + image_path
# featured_image_url
mars
# image_path

{'news_title': "What's Mars Solar Conjunction, and Why Does It Matter?",
 'news_p': 'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01322_hires.jpg'}

In [7]:
# Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts 
# about the planet including Diameter, Mass, etc.

url3 = 'https://space-facts.com/mars/'
browser.visit(url3)
html = browser.html
soup = bs(html, "html.parser")

In [8]:
tables = pd.read_html(url3)
tables

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [9]:
df=tables[1]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [10]:
renamed_df = df.rename(columns={
    0: "Description",
    1: "Value"
})
renamed_df.head()

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [11]:
# Use Pandas to convert the data to a HTML table string.
renamed_df.to_html('mars_facts.html')

In [41]:
url4='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars' 
browser.visit(url4)

hemisphere_image_urls =[]

for i in range(4):
    mars_title_and_images={}  

    html = browser.html
    soup = bs(html, "html.parser")

    titles = soup.find_all('h3')
    mars_title_and_images['title']= titles[i].get_text().strip()

    image= browser.find_by_tag('h3')[i]
    image.click()

    html = browser.html
    soup = bs(html, "html.parser")

    mars_title_and_images['image_url']= soup.find('li').find('a')['href']


    hemisphere_image_urls.append(mars_title_and_images)
    browser.back()
    
hemisphere_image_urls 

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [42]:
browser.quit()

In [43]:
!jupyter nbconvert --to script scrape_mars.ipynb

[NbConvertApp] Converting notebook scrape_mars.ipynb to script
[NbConvertApp] Writing 7250 bytes to scrape_mars.py
